In [1]:
# Importing Library
import numpy as np
from astropy.io import fits
from matplotlib import pyplot as plt
from astropy.coordinates import Angle
from astropy import units as u

In [2]:
from gaus_generator import source

In [3]:
#loading Fits File
filename = 'etest_cube.fits'
hdul = fits.open(filename)
hdul.info()

Filename: etest_cube.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      78   (512, 512, 65, 1)   float32   
  1  BEAMS         1 BinTableHDU     30   65R x 5C   [1E, 1E, 1E, 1J, 1J]   


In [4]:
hdr = hdul[0].header
len(hdr)

78

In [5]:
cdel = Angle(hdr['CDELT1'],u.degree)
dmaj = Angle(hdr['BMAJ'],u.degree)

In [7]:
cdel_1 = Angle(hdr['CDELT1'],u.degree)      #pixel increment from header
dmaj = Angle(hdr['BMAJ'],u.degree)          #BMAJ FWHM from header

c_1 = np.abs(cdel_1.arcsec)
d_1= dmaj.arcsec

cdel_2 = Angle(hdr['CDELT2'],u.degree)      #pixel increment from header
dmIN = Angle(hdr['BMIN'],u.degree)          #BMIN FWHM from header

c_2 = np.abs(cdel_1.arcsec)
d_2 = dmIN.arcsec

t_x = d_1/c_1;t_y = d_2/c_2

# Number of Spanning Channells
r = 8
#r = input('Enter the Number Channels to Span: ')
r = int(r)

In [9]:
data = hdul[0].data[0]

In [10]:
source(data,t_x,t_y,r)

array([[[0.01242016, 0.01253232, 0.01260765, 0.01264549, 0.01264549,
         0.01260765, 0.01253232, 0.01242016],
        [0.01251946, 0.01263251, 0.01270845, 0.01274659, 0.01274659,
         0.01270845, 0.01263251, 0.01251946],
        [0.01259724, 0.012711  , 0.01278741, 0.01282578, 0.01282578,
         0.01278741, 0.012711  , 0.01259724],
        [0.01265309, 0.01276735, 0.0128441 , 0.01288265, 0.01288265,
         0.0128441 , 0.01276735, 0.01265309],
        [0.01268672, 0.01280129, 0.01287824, 0.01291689, 0.01291689,
         0.01287824, 0.01280129, 0.01268672],
        [0.01269795, 0.01281262, 0.01288964, 0.01292832, 0.01292832,
         0.01288964, 0.01281262, 0.01269795],
        [0.01268672, 0.01280129, 0.01287824, 0.01291689, 0.01291689,
         0.01287824, 0.01280129, 0.01268672],
        [0.01265309, 0.01276735, 0.0128441 , 0.01288265, 0.01288265,
         0.0128441 , 0.01276735, 0.01265309],
        [0.01259724, 0.012711  , 0.01278741, 0.01282578, 0.01282578,
         0.

In [11]:
gs

NameError: name 'gs' is not defined

In [33]:
np.max(data)

0.0032292525

In [38]:
#Generating 3D Gaussian Source
r = 32
mu = t/2
mu_z = r/2
b_x = t/300
b_y = t/200
b_z = r/640
x,y,z = np.meshgrid(np.linspace(0,t,int(t)),np.linspace(0,t,int(t)),np.linspace(0,r,r))
Amp = 4*np.max(data)
gs = Amp*np.exp(-(((x-mu)**2)/2*b_x**2+((y-mu)**2)/2*b_y**2)+((z-mu_z)**2/2*b_z**2))
gs = np.transpose(gs,(2,0,1))
print(gs.shape)

(32, 11, 11)


In [46]:
# Entering the Source position
ns = input('Enter the Number of Sources:')
ns = int(ns)
sources = []
for i in range(ns):
    x_1 = input('Enter x value of Source {}: '.format(i+1))
    y_1 = input('Enter y value of Source {}: '.format(i+1))
    z_1 = input('Enter z value of Source {}: '.format(i+1))
    x_1 = float(x_1)
    y_1 = float(y_1)
    z_1 = float(z_1)
    pos = (x_1,y_1,z_1)
    sources.append(pos)
sources

Enter the Number of Sources:1
Enter x value of Source 1: 23
Enter y value of Source 1: 34
Enter z value of Source 1: 54


[(23.0, 34.0, 54.0)]

In [45]:
sources

[(34.0, 34.0), (34.0, 54.0)]

In [39]:
#Adding sources to the Cube
for source in sources:
    x,y,z = source
    y_1 = int(y-mu)
    y_2 = int(y+mu)
    x_1 = int(x-mu)
    x_2 = int(x+mu)
    z_1 = int(z-mu_z)
    z_2 = int(z+mu_z)
    arr = data[z_1:z_2,y_1:y_2,x_1:x_2]
    arr+=gs


In [40]:
hdul.writeto('new.fits',overwrite=True)